In [12]:
from shapely import wkt
from shapely.geometry import Polygon
import h3
from pathlib import Path
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
from copy import copy

In [28]:
def hex_to_poly(hex_id: str) -> Polygon:
    coords = h3.cell_to_boundary(hex_id)
    polygon = Polygon([(t[1], t[0]) for t in coords])
    return polygon

def hex_to_res(hex_id: str, target_res: int) -> str:
    out_id = copy(hex_id)
    res = h3.get_resolution(hex_id)
    print(res)
    if res < target_res:
        raise Exception(f"cannot go from h3 resolution {res} to {target_res}")
    if res == target_res:
        return out_id

    while res > target_res:
        print(f"res: {res}, id: {out_id}")
        out_id = h3.cell_to_parent(out_id)
        res = h3.get_resolution(out_id)
    return out_id

In [44]:
start = '88268cd135fffff'
h3.get_resolution(start)
hex_to_res('87268cd13ffffff', 4)

7
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff


'84268cdffffffff'

In [4]:
path = Path("/Users/rfitzger/data/mep/mep3/output/2025-06-13-bambam-test")
df = pd.read_csv(path / "result.csv")

In [5]:
df["geometry"] = df.grid_id.apply(hex_to_poly)

In [7]:
df.columns

Index(['grid_id', 'isochrone_10', 'isochrone_20', 'isochrone_30',
       'isochrone_40', 'lat', 'lon', 'mep', 'mep_entertainment', 'mep_food',
       'mep_healthcare', 'mep_jobs', 'mep_retail', 'mep_services', 'mode',
       'opps_entertainment_10', 'opps_entertainment_20',
       'opps_entertainment_30', 'opps_entertainment_40',
       'opps_entertainment_total', 'opps_food_10', 'opps_food_20',
       'opps_food_30', 'opps_food_40', 'opps_food_total', 'opps_healthcare_10',
       'opps_healthcare_20', 'opps_healthcare_30', 'opps_healthcare_40',
       'opps_healthcare_total', 'opps_jobs_10', 'opps_jobs_20', 'opps_jobs_30',
       'opps_jobs_40', 'opps_jobs_total', 'opps_retail_10', 'opps_retail_20',
       'opps_retail_30', 'opps_retail_40', 'opps_retail_total',
       'opps_services_10', 'opps_services_20', 'opps_services_30',
       'opps_services_40', 'opps_services_total', 'ram_mb', 'runtime',
       'geometry'],
      dtype='object')

In [ ]:
cols = ['grid_id', 'mep', 'mep_entertainment', 'mep_food',
       'mep_healthcare', 'mep_jobs', 'mep_retail', 'mep_services', 'mode',
       'geometry']
gdf = gpd.GeoDataFrame(df[cols].copy(), crs="EPSG:4326")
gdf["search_id"] = gdf.grid_id.apply(lambda h: hex_to_res(h, 4))
partitions = {
    "org": "test",
    "year": 2025,
    "batch": "mep_tomtom_overture",
    "scenario": "usa",
    "variant": "baseline"
}
for k, v in partitions.items():
    gdf[k] = v

9
res: 9, id: 89268cd135bffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res: 9, id: 89268cd135bffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res: 9, id: 89268cd134bffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res: 9, id: 89268cd1343ffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res: 9, id: 89268cdac87ffff
res: 8, id: 88268cdac9fffff
res: 7, id: 87268cdacffffff
res: 6, id: 86268cdafffffff
res: 5, id: 85268cdbfffffff
9
res: 9, id: 89268cd134bffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res: 9, id: 89268cd1343ffff
res: 8, id: 88268cd135fffff
res: 7, id: 87268cd13ffffff
res: 6, id: 86268cd17ffffff
res: 5, id: 85268cd3fffffff
9
res:

In [47]:
index = [
    "org",
    "year",
    "batch",
    "scenario",
    "variant",
    "search_id"
]
gdf = gdf.set_index(index)

In [ ]:
gdf.to_parquet("/Users/rfitzger/data/mep/mep3/output/2025-06-13-bambam-test/data.parquet")

In [54]:
gdf

grid_id  \
org  year batch               scenario variant  search_id                          
test 2025 mep_tomtom_overture usa      baseline 84268cdffffffff  89268cd135bffff   
                                                84268cdffffffff  89268cd135bffff   
                                                84268cdffffffff  89268cd134bffff   
                                                84268cdffffffff  89268cd1343ffff   
                                                84268cdffffffff  89268cdac87ffff   
...                                                                          ...   
                                                84268cdffffffff  89268cda8d7ffff   
                                                84268cdffffffff  89268cdac23ffff   
                                                84268cdffffffff  89268cdac2bffff   
                                                84268cdffffffff  89268cda893ffff   
                                                84268cdffffffff  89268cda89bffff   

                                                                        mep  \
org  year batch               scenario variant  search_id                     
test 2025 mep_tomtom_overture usa      baseline 84268cdffffffff    0.067641   
                                                84268cdffffffff    0.067641   
                                                84268cdffffffff    0.072068   
                                                84268cdffffffff    0.190272   
                                                84268cdffffffff    0.105777   
...                                                                     ...   
                                                84268cdffffffff  120.359903   
                                                84268cdffffffff  119.920871   
                                                84268cdffffffff  119.375210   
                                                84268cdffffffff  119.945912   
                                                84268cdffffffff  119.331186   

                                                                 mep_entertainment  \
org  year batch               scenario variant  search_id                            
test 2025 mep_tomtom_overture usa      baseline 84268cdffffffff           0.000000   
                                                84268cdffffffff           0.000000   
                                                84268cdffffffff           0.000000   
                                                84268cdffffffff           0.000000   
                                                84268cdffffffff           0.014245   
...                                                                            ...   
                                                84268cdffffffff          30.007279   
                                                84268cdffffffff          29.219686   
                                                84268cdffffffff          28.904649   
                                                84268cdffffffff          29.928519   
                                                84268cdffffffff          29.219686   

                                                                 mep_food  \
org  year batch               scenario variant  search_id                   
test 2025 mep_tomtom_overture usa      baseline 84268cdffffffff  0.000000   
                                                84268cdffffffff  0.000000   
                                                84268cdffffffff  0.000000   
                                                84268cdffffffff  0.000000   
                                                84268cdffffffff  0.000000   
...                                                                   ...   
                                                84268cdffffffff  0.487096   
                                                84268cdffffffff  0.514157   
                                                84268cdffffffff